Disclaimer: this following code is not owned by me. 

In [26]:
# 045dc66ba8683c1ae95c3dc9aa812780a283315aca280dab8c242a20cfa2f19d

In [27]:
!pip install cryptocompy

  Preparing metadata (setup.py) ... done
  Created wheel for cryptocompy: filename=cryptocompy-0.1.1.dev1-py3-none-any.whl size=7483 sha256=c83c047d912fa529f510f8a6fbc0760f48fe60f3646980be22f692510f051feb
  Stored in directory: /home/liyam/.cache/pip/wheels/84/af/b2/163885401222dd97392a1d2ade7c8f1869f837b0e49cf0e345
Successfully built cryptocompy


In [53]:
from cryptocompy import coin
symbol_list = list(coin.get_coin_list(coins='all').keys())
symbol_list

['BTC',
 'ETH',
 'LTC',
 'DASH',
 'XMR',
 'NXT',
 'ETC',
 'DOGE',
 'ZEC',
 'BTS',
 'DGB',
 'XRP',
 'BTCD',
 'PPC',
 'CRAIG',
 'XBS',
 'XPY',
 'PRC',
 'YBC',
 'DANK',
 'GIVE',
 'KOBO',
 'DT',
 'CETI',
 'SUP',
 'XPD',
 'GEO',
 'CHASH',
 'NXTI',
 'INSANITY',
 'XDP',
 '2015',
 '42',
 'AC',
 'ACOIN',
 'AEROCOIN',
 'ALF',
 'AEGIS',
 'AMERICANCOIN',
 'ALIEN',
 'APEXCOIN',
 'ARCHCOIN',
 'ARGENTUM',
 'ARI',
 'AURORAC',
 'AXR',
 'BETACOIN',
 'BITB',
 'BLU',
 'BLK',
 'BOST',
 'BQC',
 'XMY',
 'MOONC',
 'ZET',
 'SXC',
 'QTL',
 'ENRG',
 'QRK',
 'RIC',
 'DGC',
 'LIMX',
 'BTB',
 'CAIX',
 'BTMK',
 'BUK',
 'CACH',
 'CANN',
 'CASH',
 'CATC',
 'CRYPTOBULLION',
 'CCN',
 'CIN',
 'CINNI',
 'CXC',
 'CLAM',
 'CLOAK',
 'CLR',
 'CMC',
 'CNC',
 'CNL',
 'COMM',
 'COOL',
 'CRACK',
 'CRYPT',
 'CSC',
 'DEM',
 'DIAM',
 'XVG',
 'DRKC',
 'DSB',
 'DEVCOIN',
 'EARTHCOIN',
 'EFL',
 'ELC',
 'EMC2',
 'EMD',
 'EXCL',
 'EXE',
 'EZC',
 'FLAP',
 'FC2',
 'FFC',
 'FIBRE',
 'FRC',
 'FRK',
 'FRAC',
 'FSTC',
 'FTC',
 'GRANDCOIN',
 'G

In [54]:
import cryptocompare
import pandas as pd
import datetime
import time
import os
from pathlib import Path
import config



import requests
from typing import Union, Optional, List, Dict
Timestamp = Union[datetime.datetime, datetime.date, int, float]

In [55]:
# Set the API key
key = "045dc66ba8683c1ae95c3dc9aa812780a283315aca280dab8c242a20cfa2f19d"
cryptocompare.cryptocompare._set_api_key_parameter(key)

'&api_key=045dc66ba8683c1ae95c3dc9aa812780a283315aca280dab8c242a20cfa2f19d'

In [56]:
# API
_URL_PRICE = 'https://min-api.cryptocompare.com/data/pricemulti?fsyms={}&tsyms={}' 
_URL_PRICE_MULTI = 'https://min-api.cryptocompare.com/data/pricemulti?fsyms={}&tsyms={}'
_URL_PRICE_MULTI_FULL = 'https://min-api.cryptocompare.com/data/pricemultifull?fsyms={}&tsyms={}'
_URL_HIST_PRICE = 'https://min-api.cryptocompare.com/data/pricehistorical?fsym={}&tsyms={}&ts={}&e={}'
_URL_HIST_PRICE_DAY = 'https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&limit={}&e={}&toTs={}'
_URL_HIST_PRICE_HOUR = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&e={}&toTs={}'
# _URL_EXCHANGES = 'https://www.cryptocompare.com/api/data/exchanges?'
_URL_EXCHANGES_HOUR = 'https://min-api.cryptocompare.com/data/exchange/histohour?tsym={}&limit={}&toTs={}'

# DEFAULTS
CURRENCY = 'USD'    # Default currency
LIMIT = 2000    # Default limit

Cryptocompare provides a Github for scraping Real-time Crypto prices. Source:
https://github.com/lagerfeuer/cryptocompare/blob/master/cryptocompare/cryptocompare.py

In [57]:

def _query_cryptocompare(url: str, errorCheck: bool = True, api_key: str = None) -> Optional[Dict]:
    """
    Query the url and return the result or None on failure.
    :param url: the url
    :param errorCheck: run extra error checks (default: True)
    :returns: respones, or nothing if errorCheck=True
    :api_key: optional, if you want to add an API Key
    """
    api_key_parameter = _set_api_key_parameter(api_key)
    try:
        response = requests.get(url + api_key_parameter).json()
    except Exception as e:
        print('Error getting coin information. %s' % str(e))
        return None
    if errorCheck and (response.get('Response') == 'Error'):
        print('[ERROR] %s' % response.get('Message'))
        return None
    return response


def _format_parameter(parameter: object) -> str:
    """
    Format the parameter depending on its type and return
    the string representation accepted by the API.
    :param parameter: parameter to format
    """
    if isinstance(parameter, list):
        return ','.join(parameter)

    else:
        return str(parameter)


def _format_timestamp(timestamp: Timestamp) -> int:
    """
    Format the timestamp depending on its type and return
    the integer representation accepted by the API.
    :param timestamp: timestamp to format
    """
    if isinstance(timestamp, datetime.datetime) or isinstance(timestamp, datetime.date):
        return int(time.mktime(timestamp.timetuple()))
    return int(timestamp)


def _set_api_key_parameter(api_key: str = None) -> str:
    if api_key is None:
        api_key = os.getenv('CRYPTOCOMPARE_API_KEY')
    if api_key is not None:
        _API_KEY = "&api_key={}".format(api_key)
        return _API_KEY
    return ""

In [14]:
def get_price(coin: str, currency: str = CURRENCY, full: bool = False) -> Optional[Dict]:
    """
    Get the currencyent price of a coin in a given currency.
    :param coin: symbolic name of the coin (e.g. BTC)
    :param currency: short hand description of the currency (e.g. EUR)
    :param full: full response or just the price (default: False)
    :returns: dict of coin and currency price pairs
    """
    if full:
        return _query_cryptocompare(
            _URL_PRICE_MULTI_FULL.format(
                _format_parameter(coin), _format_parameter(currency))
        )
    if isinstance(coin, list):
        return _query_cryptocompare(
            _URL_PRICE_MULTI.format(_format_parameter(coin),
                                    _format_parameter(currency))
        )
    return _query_cryptocompare(
        _URL_PRICE.format(coin, _format_parameter(currency))
    )


def get_historical_price(coin: str, currency: str = CURRENCY, timestamp: Timestamp = time.time(),
                         exchange: str = 'CCCAGG') -> Optional[Dict]:
    """
    Get the price of a coin in a given currency during a specific time.
    :param coin: symbolic name of the coin (e.g. BTC)
    :param currency: short hand description of the currency (e.g. EUR)
    :param timestamp: point in time
    :param exchange: the exchange to use
    :returns: dict of coin and currency price pairs
    """
    return _query_cryptocompare(
        _URL_HIST_PRICE.format(coin,
                               _format_parameter(currency),
                               _format_timestamp(timestamp),
                               _format_parameter(exchange))
    )


def get_historical_price_day(coin: str, currency: str = CURRENCY, limit: int = LIMIT,
                             exchange: str = 'CCCAGG', toTs: Timestamp = time.time()) -> Optional[Dict]:
    """
    Get historical price (day).
    :param coin: symbolic name of the coin (e.g. BTC)
    :param currency: short hand description of the currency (e.g. EUR)
    :param limit: number of data points (max. 2000)
    :param exchange: exchange to use (default: 'CCCAGG')
    :param toTs: return data before this timestamp. (Unix epoch time or datetime object)
    :returns: dict of coin and currency price pairs
    """
    response = _query_cryptocompare(
        _URL_HIST_PRICE_DAY.format(coin, _format_parameter(currency), limit, exchange, _format_timestamp(toTs)))
    if response:
        return response['Data']
    return None


def get_historical_price_hour(coin: str, currency: str = CURRENCY, limit: int = LIMIT,
                              exchange: str = 'CCCAGG', toTs: Timestamp = time.time()) -> Optional[Dict]:
    """
    Get historical price (hourly).
    :param coin: symbolic name of the coin (e.g. BTC)
    :param currency: short hand description of the currency (e.g. EUR)
    :param limit: number of data points (max. 2000)
    :param exchange: exchange to use (default: 'CCCAGG')
    :param toTs: return data before this timestamp. (Unix epoch time or datetime object)
    :returns: dict of coin and currency price pairs
    """
    response = _query_cryptocompare(
        _URL_HIST_PRICE_HOUR.format(coin, _format_parameter(currency), limit, exchange, _format_timestamp(toTs)))
    if response:
        return response['Data']
    return None


def get_exchanges(currency: str = CURRENCY, limit: int = LIMIT, toTs: Timestamp = time.time()) -> Optional[Dict]:
    """
    Get the list of available exchanges.
    :returns: list of available exchanges
    """
    response = _query_cryptocompare(_URL_EXCHANGES_HOUR.format(currency, limit, _format_timestamp(toTs)))
    if response:
        return response['Data']
    return None

### Scraping crypto prices for Price prediction 

In [69]:

for symbol in symbol_list:
    path = Path(f'{os.getcwd()}')

    # CHOOSE ONE OF THE FOLLOWING 
    # 1. GET THE PRICE OF A COIN IN A GIVEN CURRENCY
    # 2. GET THE PRICE OF A COIN IN A GIVEN CURRENCY AT A SPECIFIC TIME
    # 3. GET THE PRICE OF A COIN IN A GIVEN CURRENCY AT A SPECIFIC TIME FOR A SPECIFIC EXCHANGE
    # 4. GET THE PRICE OF A COIN IN A GIVEN CURRENCY FOR A SPECIFIC EXCHANGE

    # df = get_historical_price_day('BTC', 'USD', limit=2000)
    # df = get_historical_price_hour('DOGE', currency='USD', limit=2000)
    df = get_historical_price_hour(symbol, currency='USD', limit=2000)
    # df_volume = get_exchanges(toTs=time_stamp)
    # Comvert df to dataframe and save to csv
    df = pd.DataFrame(df)
    # df = df[['time', 'close']]
    # df = df[['time', 'volume']]
    # df['time'] = pd.to_datetime(df['time'], unit='s')

    
    # df = df.sort_values(by=['time'], ascending=False)
    filename = symbol+".csv"
    df.to_csv(filename)

[ERROR] CCCAGG market does not exist for this coin pair (BTCD-USD)
[ERROR] CCCAGG market does not exist for this coin pair (CRAIG-USD)
[ERROR] CCCAGG market does not exist for this coin pair (XBS-USD)
[ERROR] CCCAGG market does not exist for this coin pair (XPY-USD)
[ERROR] CCCAGG market does not exist for this coin pair (PRC-USD)
[ERROR] CCCAGG market does not exist for this coin pair (YBC-USD)
[ERROR] CCCAGG market does not exist for this coin pair (DANK-USD)
[ERROR] CCCAGG market does not exist for this coin pair (GIVE-USD)
[ERROR] CCCAGG market does not exist for this coin pair (KOBO-USD)
[ERROR] CCCAGG market does not exist for this coin pair (DT-USD)
[ERROR] CCCAGG market does not exist for this coin pair (CETI-USD)
[ERROR] CCCAGG market does not exist for this coin pair (SUP-USD)
[ERROR] CCCAGG market does not exist for this coin pair (XPD-USD)
[ERROR] CCCAGG market does not exist for this coin pair (CHASH-USD)
[ERROR] CCCAGG market does not exist for this coin pair (NXTI-USD)
[

In [59]:
# Comvert df to dataframe and save to csv
df = pd.DataFrame(df)
# df = df[['time', 'close']]
# df = df[['time', 'volume']]
# df['time'] = pd.to_datetime(df['time'], unit='s')


df = df.sort_values(by=['time'], ascending=False)

In [22]:
df.to_csv(f"DOGE-price_history-raw.csv")